In [4]:
import cv2 as cv


In [18]:
import cv2
import mediapipe as mp
import numpy as np
from collections import deque

def run_height_detection():

    try:
        shoulder_width_input = input("Please enter your shoulder width in cm (e.g., 40.0) and press Enter: ")
        KNOWN_SHOULDER_WIDTH_CM = float(shoulder_width_input)
    except (ValueError, TypeError):
        print("Invalid input. Using default shoulder width of 40.0 cm.")
        KNOWN_SHOULDER_WIDTH_CM = 40.0

   
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(
        static_image_mode=False, 
        min_detection_confidence=0.5, 
        min_tracking_confidence=0.5
    )
    mp_drawing = mp.solutions.drawing_utils
    
    height_history = deque(maxlen=10)

    def check_posture(landmarks):
        """Checks if the person is standing straight based on shoulder-hip alignment."""
        try:
            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
            left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
            right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]

            
            if all(lm.visibility > 0.7 for lm in [left_shoulder, right_shoulder, left_hip, right_hip]):
                shoulder_mid_x = (left_shoulder.x + right_shoulder.x) / 2
                hip_mid_x = (left_hip.x + right_hip.x) / 2
                
                
                if abs(shoulder_mid_x - hip_mid_x) < 0.05:
                    return True
        except Exception:
            return False
        return False

    def estimate_height(landmarks, image_height, image_width):
        """Estimates height only when posture is correct."""
        try:
            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
            top_point = landmarks[mp_pose.PoseLandmark.NOSE.value]
            left_heel = landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value]
            right_heel = landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value]
            bottom_point_y = (left_heel.y + right_heel.y) / 2

           
            if all(lm.visibility > 0.6 for lm in [left_shoulder, right_shoulder, top_point, left_heel, right_heel]):
                shoulder_width_pixels = np.sqrt(
                    ((left_shoulder.x - right_shoulder.x) * image_width)**2 +
                    ((left_shoulder.y - right_shoulder.y) * image_height)**2
                )
                body_height_pixels = (bottom_point_y - top_point.y) * image_height
                
                if shoulder_width_pixels == 0:
                    return None
                
                estimated_height_cm = (body_height_pixels / shoulder_width_pixels) * KNOWN_SHOULDER_WIDTH_CM
                return estimated_height_cm
        except Exception as e:
            return None
        return None

    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("Error: Cannot open camera.")
        return

    print("Starting camera... A new window will open.")
    print(f"Using a shoulder width of {KNOWN_SHOULDER_WIDTH_CM} cm for calibration.")
    print("Press 'q' to exit.")
    
    try:
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
                continue

            image = cv2.flip(image, 1)
            image_height, image_width, _ = image.shape
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = pose.process(image_rgb)

            if results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                )
                
                is_straight = check_posture(results.pose_landmarks.landmark)

                if is_straight:
                    height_cm = estimate_height(results.pose_landmarks.landmark, image_height, image_width)
                    if height_cm is not None:
                        height_history.append(height_cm)
                    
                    if height_history:
                        smoothed_height = np.mean(height_history)
                        display_text = f"Estimated Height: {smoothed_height:.2f} cm"
                        cv2.putText(image, display_text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                else:
                    
                    cv2.putText(image, "Please stand straight", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                    height_history.clear()

            cv2.imshow('MediaPipe Height Detection', image)
            
            if cv2.waitKey(5) & 0xFF == ord('q'):
                break

    finally:
        print("Finished.")
        cap.release()
        cv2.destroyAllWindows()
        pose.close()



In [20]:
run_height_detection()

Starting camera... A new window will open.
Using a shoulder width of 40.0 cm for calibration.
Press 'q' to exit.
Finished.
